In [28]:
from scipy.stats import norm
from time import time
import random    
import pandas as pd

# -1) Cycliste

In [29]:
class logger():
    """
    Save data into memory (in an array) and write it down to a file after [nb_log] is reached.
    """
    def __init__(self, path, limit_to_save_to_disk=100):
        self.path    = path
        self.limit   = limit_to_save_to_disk
        self.logs    = []
        self.nb_logs = 0
        
    def log(self, string):
        self.logs.append(string)
        self.nb_logs = self.nb_logs +1
        if self.limit < self.nb_logs:
            self.write_to_disk()
    
    def write_to_disk(self):
        with open(self.path, "a") as f:
            while len(self.logs):
                try:
                    a = self.logs.pop()
                    f.write(u"%s\n"%a.decode('ascii', errors='ignore'))
                except:
                    print "pbm with "
                    print a
        self.nb_logs = len(self.logs)
    def __del__(self):
        self.write_to_disk()
        

In [30]:
class Cycliste:
    

            
        
    def __init__(self, identifiant):
        import time
        from scipy.stats import norm
        import random
        homme   = "homme"
        femme   = "femme"
        age_min = 15
        age_max = 80
        sexe    = random.choice([homme, femme])
        age     = random.randint(age_min ,age_max)
        sportif = random.choice([-0.5, 0, 2, 4, 6 ])
        if age < 35:
            facteur = norm( loc =1.2, scale=0.2 )
        elif age <50:
            facteur = norm( loc =1  , scale=0.2 )
        elif age <90:
            facteur = norm( loc =0.7, scale=0.2 )

        if sexe == homme:
            vitesse_de_base = 20
            nb_km = norm( loc =7, scale=2 )
        else:
            vitesse_de_base = 15
            nb_km = norm( loc =10, scale=2 )
        vitesse_moyenne = vitesse_de_base * facteur.rvs(1) + sportif
        
        self.sportif     = sportif
        self.age         = age
        self.sexe        = sexe
        self.nb_km       = nb_km.rvs(1)[0]
        self.vitesse     = vitesse_moyenne[0]
        self.sur_velo    = False
        self.debug       = False
        self.nb_trajet   = 0
        self.durees      = []
        self.trajets     = []
        self.velo        = False
        self.identifiant = "cycliste_%s"%identifiant
        self.prises      = []
        self.attente         = random.randint(0,100)
        self.last_rendu      = time.time()
        self.logger_cycliste = logger("./logs/cycliste_cyclistes.csv")
        self.logger_prise    = logger("./logs/cycliste_prises.csv")
        self.logger_rendu    = logger("./logs/cycliste_rendu.csv")
        self.logger_debug    = logger("./logs/cycliste_debug.csv")
        
        self.log_cycliste()
        
    def log_debug(self, message):
        from time import time
        to_print = u"%s,%s,%s"% (self.identifiant, time() , message)
        self.logger_debug.log(to_print)
    
    def peut_prendre_velo(self, station):
        """
        Renvoie un velo ou bien False.
        """
        from time import time
        if time() < self.attente + self.last_rendu:
            self.log_debug(u"trop tot")
            return False
        if self.sur_velo:
            self.log_debug(u"deja sur velo")
            return False
        velo = station.prendre_velo()
        if not velo:
            self.log_debug(u"pas de velo sur la station %s " % station)
            return False
        return velo
        
    def prend_velo(self, station):
        import time
        
        velo = self.peut_prendre_velo(station)
        if not velo :
            return
        
        velo.disponible      = False
        self.sur_velo        = True
        self.heure_de_depart = time.time()
        self.velo            = velo
        self.log_prise()
        try:
            nb_km_local          = norm( loc =self.nb_km, scale=self.nb_km/float(3) ).rvs(1)[0]
        except Exception as e:
            self.log_debug(u"pbm sur nb_km_local = self.nb_km = %s " %self.nb_km)
            nb_km_local = 15
        try:
            vitesse_local        = norm(loc = self.vitesse, scale = self.vitesse/float(10)).rvs(1)[0]
            vitesse_local        = vitesse_local * velo.performance
        except:
            self.log_debug(u"pbm sur vitesse_local self.vitesse = %s"%self.vitesse)
            vitesse_local = self.vitesse
            
        self.nb_km_trajet    = nb_km_local
        duree                = nb_km_local / vitesse_local
        self.heure_d_arrivee = self.heure_de_depart + duree
        
        if velo.performance <0.1:
            duree                = 0
            self.durees.append( duree)
            self.heure_d_arrivee =  self.heure_de_depart
            self.nb_km_trajet    = 0
            self.rendre_velo(station)
        elif velo.performance <0.3:
            duree                = duree / 10
            self.durees.append( duree)
            self.heure_d_arrivee =  self.heure_de_depart + duree
            self.nb_km_trajet    = self.nb_km_trajet / float(10)
            self.rendre_velo(station)
        elif velo.performance <0.5:
            duree                = duree / 5
            self.durees.append( duree)
            self.nb_km_trajet    = self.nb_km_trajet / float(5)
            self.heure_d_arrivee =  self.heure_de_depart + duree 
        else:
            self.durees.append( duree)




    def rendre_velo(self, station):        
        import time
        if not self.sur_velo:
            if self.debug : print u"pas en chemin",
            return 0
        if time.time() > self.heure_d_arrivee:
            if station.rendre_velo(self.velo, self.nb_km_trajet):
                
                #if self.debug : print "plus sur velo",
                duree_constatee = self.heure_d_arrivee - self.heure_de_depart
                trajet          = (self.heure_de_depart, self.heure_d_arrivee, self.identifiant, self.nb_km_trajet, self.velo.nom )
                self.log_rendu()
                self.last_rendu = time.time()
                self.sur_velo  = False
                self.nb_trajet = self.nb_trajet + 1
                self.velo      = False
                self.trajets.append(trajet)
                
            else:
                if self.debug : print u"impossible de rendre sur station %s"%(station),
                pass
        else:
            if self.debug : print u"pas encore arrive",
            pass
        
    def log_cycliste(self):
        to_print = u"%s,%s,%s,%s,%s,%s,%s"%(    self.identifiant   ,self.sportif     ,
                                                 self.age         ,self.sexe        ,
                                                 self.nb_km       ,self.vitesse     , self.attente )
        
        self.logger_cycliste.log(to_print)
            
    def log_prise(self, action="prise"):
        to_print = u"%s,%s,%s,%s"%( self.identifiant,self.heure_de_depart, self.velo.nom, action)
        self.logger_prise.log(to_print)
            
    def log_rendu(self, action="prise"):
        to_print = u"%s,%s,%s,%s,%s\n"%( self.identifiant, self.heure_de_depart, 
                                       self.heure_d_arrivee,  self.nb_km_trajet, self.velo.nom )

        self.logger_rendu.log(to_print)


# -1) Velo

In [31]:
class Velo:

    def __init__(self, nom, station):
        from scipy.stats import norm
        import time 
        self.nom          = nom
        performance       = norm( loc =0.9, scale=0.2 )
        self.performance  = performance.rvs(1)[0]
        self.station      = station
        self.degradation  = 0
        self.performances = []
        self.debug        = False
        self.nb_km_trajet = 0
        self.heures_rendu = []
        self.disponible   = True
        self.logger_reparation = logger("./logs/velo_reparations.csv")
        self.logger_etat       = logger("./logs/velos_etats.csv")
        self.probleme_list     = self.get_problemes_list()

    def get_problemes_list(self):
        """
        Créé le tableau de dégradations possible pour un vélo.
        """
        problemes       = [0] * 50
        problemes_bis   = {u"pedale" : 0.2, u"roue"  : 1 , u"degonfle" : 0.3, u"freins" : 0.05 , 0 : 0}
        problemes.extend(problemes_bis.values())
        return problemes
        
    def log_reparation(self):
        import time
        reparation =(self.nom,time.time(),self.performance)
        to_print = u"%s,%s,%s"%reparation
        self.logger_reparation.log(to_print)
        

    def log_etat(self):
        from time import time
        to_print = u"%s,%s,%s,%s,%s\n"%(self.nom           ,
                                           time()            ,  
                                           self.station.nom  ,
                                           self.performance  ,
                                           self.nb_km_trajet )
        self.logger_etat.log(to_print)
    
    def rendu(self, station, nb_km_trajet):
        import time
        self.disponible = True
        if self.debug : print(u"velo %s rendu sur station %s"%(self, station))
            
        degradation       = random.choice(self.probleme_list)        
        self.performance  = self.performance - self.performance*degradation                                        
        self.nb_km_trajet = self.nb_km_trajet + nb_km_trajet
        self.station = station
        self.log_etat()
        if 900 < random.randint(0, 1000 ) :
            self.performance  = 1
            self.log_reparation()
        
        

    def __str__(self):
        return "%s"%self.nom



# -1 ) Station

In [32]:
class Station:
    def __init__(self, nom, debug=False ):
        import random
        import time
        debut         = time.time()
        self.nom      = nom
        self.nb_plot  = random.choice([10,15,20, 30])
        self.nb_libre = 0 #random.randint(0, self.nb_plot )
        self.nb_velos = self.nb_plot # - self.nb_libre
        self.velos    = []
        noms          = get_names()
        
        for n in range(self.nb_velos ):
            nom_velo= u"velo_"+"".join(["%s"%x for x in random.choice(noms)])
            self.velos.append(Velo(nom_velo , self))
        
        if debug : print u"created %s velos"%len(self.velos),
        self.debug          = False
        self.avaries        = []
        self.arrete         = False
        duree               = time.time() - debut
        if debug : print u"station créée en %s"%duree
        
        self.logger_dispos = logger("./logs/stations_dispos.csv")
        self.logger_reparation = logger("./logs/stations_reparations.csv")
        self.logger_avarie = logger("./logs/stations_avarie.csv")
        
        self.liste_d_avaries = self.get_avarie_list()
        self.log_dispo()


    def log_reparation(self):
        from time import time
        dispo =(self.nom, time(),self.nb_plot, self.nb_libre, self.nb_velos)
        to_print = u"%s,%s,%s,%s,%s"%dispo
        self.logger_reparation.log(to_print)
            
            
    def log_dispo(self):
        from time import time
        dispo =(self.nom,time(),self.nb_plot, self.nb_libre, self.nb_velos)
        to_print = u"%s,%s,%s,%s,%s"%dispo
        self.logger_dispos.log(to_print)
            
    def log_avarie(self, avarie):
        from time import time
        to_print = u"%s,%s,%s"%(self.nom, time(), avarie )
        self.logger_avarie.log(to_print)

        
    def get_avarie_list(self):
        avaries     = [0]*100000
        plot        = u"plot mort"
        half        = u"50% mort"
        network     = u"reseau"
        electricite = u"eteint"
        bug         = u"bug"
        problemes   = { plot       : 20 ,
                       half        : 10 ,
                       network     :  5 ,
                       electricite : 30 , 
                       bug         :  3 }
        
        for k,v in problemes.iteritems():
            avaries.extend([k] * v)
        return avaries
    
    def avarie_possible(self):
        import math
        import random
        import time
        
        plot        = u"plot mort"
        half        = u"50% mort"
        network     = u"reseau"
        electricite = u"eteint"
        bug         = u"bug"
        
        avaries = list(self.liste_d_avaries)
        if plot in self.avaries : avaries.extend( [plot] * 60 )
        if half in self.avaries : avaries.extend( [half] *100 )
        
        avarie = random.choice(avaries)
        self.avaries.extend([avarie])
        if avarie == plot :
            self.nb_libre = self.nb_libre -1
        elif avarie == half:
            self.nb_libre = math.floor(self.nb_libre - (self.nb_plot / float(2)))
        elif avarie == network :
            self.nb_libre = 0
            self.arrete = True
        elif avarie == electricite :
            self.nb_libre = 0
            self.arrete = True
        elif avarie == bug :
            if self.nb_libre >0:
                self.nb_libre = self.nb_libre - random.randint(0, self.nb_libre)
        if self.nb_libre <0:
            self.nb_libre = 0            
        self.log_dispo()
        
        if avarie:
            self.log_avarie("%s"%avarie)
            
    def rendre_velo(self, velo, nb_km_trajet):
        import time
        self.avarie_possible()
        if self.nb_libre >0:
            self.velos.append(velo)
            velo.rendu(self, nb_km_trajet)
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre - 1
            self.log_dispo()
            return 1
        if 9500 < random.randint(0, 10000 ) :
            self.nb_libre = self.nb_plot - self.nb_velos
            self.log_reparation()
            
        return 0

    def prendre_velo(self):
        """
        retourne un vélo ou 0 en cas d'erreur.
        """
        import time
        self.avarie_possible()
        if self.velos and not self.arrete :
            to_return     = self.velos.pop()
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre + 1
            self.log_dispo()
            return to_return

        return 0
    def __str__(self):
        return u"station %s : self.nb_libre = %s, self.nb_velos =%s "%(self.nom,self.nb_libre, self.nb_velos)


# 0) get_names

In [33]:
def get_names(nb_lettres=5, nb_noms = 15000, prefix=""):
    """
    retourne n mot de nb_lettres avec prefix.
    """
    import itertools
    from random import shuffle
    lettres = "azertyuiopqsdfghjklmwxcvbn1234567890"
    names   = []
    for i in itertools.combinations(lettres, nb_lettres):
        name = "".join(i)
        name = prefix + name
        names.append(name)
        nb_noms = nb_noms -1
        if nb_noms <2:
            break
    shuffle(names)
    return names

# 0) création des stations

In [34]:
from time import time
import multiprocessing
n = 4
debut = time()
process_pool = multiprocessing.Pool(n)
debut        = time()
names        = get_names()
noms         = [names[i] for i in range(100)]
stations     = process_pool.map(Station, noms)
nb_velos     = sum([len(station.velos) for station in stations])
duree = time() - debut
print "duree = %s pour %s cree soit %s sec / unite"%(duree, nb_velos, duree/nb_velos)


duree = 37.7279729843 pour 1865 cree soit 0.020229476131 sec / unite


# Creation des cycliste

In [35]:
n = 4
process_pool = multiprocessing.Pool(n)
names        = get_names(5, nb_velos*2)
noms         = [names[i] for i in range(nb_velos)]
del names
cyclistes   = process_pool.map(Cycliste, noms)
print "duree = %s pour %s cree soit %s sec / unite"%(duree, len(cyclistes), duree/nb_velos)


duree = 37.7279729843 pour 1865 cree soit 0.020229476131 sec / unite


# voyages

In [36]:
from time import time
import multiprocessing
import math
from time import time

In [ ]:
!rm ./logs/*

In [ ]:
depart = time()
n = 0  
while time() < depart + 3600 : 
    a         = time()
    old_nb_trajet = sum([cycliste.nb_trajet for cycliste in cyclistes])
    en_route = filter(lambda cycliste:     cycliste.sur_velo, cyclistes)
    a_pied   = filter(lambda cycliste: not cycliste.sur_velo, cyclistes)
    map(lambda cycliste: cycliste.rendre_velo(random.choice(stations)), en_route  )
    map(lambda cycliste: cycliste.prend_velo(random.choice(stations)),  a_pied    )
    new_nb_trajet =  sum([cycliste.nb_trajet for cycliste in cyclistes]) - old_nb_trajet
    duree_totale = time()-a
    print "(%03d)%03d/% 3.2f|"%(n, new_nb_trajet, duree_totale),

    n += 1
print "*"*30
print "fini"
print "*"*30

(000)000/ 9.33| (001)039/ 0.04| (002)007/ 0.02| (003)008/ 0.02| (004)006/ 0.02| (005)008/ 0.02| (006)005/ 0.02| (007)005/ 0.02| (008)007/ 0.02| (009)007/ 0.02| (010)009/ 0.02| (011)005/ 0.02| (012)003/ 0.02| (013)005/ 0.01| (014)002/ 0.02| (015)007/ 0.02| (016)006/ 0.02| (017)004/ 0.02| (018)008/ 0.02| (019)005/ 0.02| (020)002/ 0.02| (021)004/ 0.02| (022)003/ 0.02| (023)005/ 0.02| (024)000/ 0.01| (025)003/ 0.02| (026)002/ 0.02| (027)003/ 0.02| (028)001/ 0.02| (029)005/ 0.02| (030)010/ 0.02| (031)004/ 0.03| (032)003/ 0.03| (033)005/ 0.02| (034)001/ 0.02| (035)003/ 0.02| (036)002/ 0.02| (037)001/ 0.02| (038)001/ 0.02| (039)002/ 0.01| (040)003/ 0.02| (041)003/ 0.02| (042)004/ 0.02| (043)002/ 0.02| (044)000/ 0.02| (045)004/ 0.02| (046)003/ 0.03| (047)006/ 0.02| (048)001/ 0.02| (049)001/ 0.02| (050)001/ 0.02| (051)001/ 0.02| (052)002/ 0.03| (053)003/ 0.03| (054)001/ 0.02| (055)001/ 0.02| (056)000/ 0.02| (057)000/ 0.02| (058)001/ 0.02| (059)002/ 0.02| (060)003/ 0.02| (061)001/ 0.01| (062)002

/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:101: RuntimeWarning: divide by zero encountered in double_scalars


 (087)001/ 0.03| (088)001/ 0.03| (089)000/ 0.02| (090)001/ 0.02| (091)001/ 0.02| (092)003/ 0.02| (093)000/ 0.03| (094)003/ 0.03| (095)002/ 0.03| (096)001/ 0.02| (097)002/ 0.02| (098)002/ 0.02| (099)002/ 0.03| (100)003/ 2.50| (101)028/ 0.13| (102)023/ 0.12| (103)007/ 0.06| (104)006/ 0.04| (105)007/ 0.05| (106)004/ 0.05| (107)008/ 0.04| (108)003/ 0.04| (109)004/ 0.04| (110)003/ 0.04| (111)003/ 0.04| (112)006/ 0.03| (113)003/ 0.03| (114)002/ 0.03| (115)004/ 0.03| (116)001/ 0.04| (117)004/ 0.04| (118)007/ 0.03| (119)002/ 0.04| (120)002/ 0.02| (121)004/ 0.03| (122)005/ 0.03| (123)003/ 0.06| (124)002/ 0.04| (125)002/ 0.03| (126)000/ 0.04| (127)005/ 0.03| (128)006/ 0.03| (129)003/ 0.04| (130)006/ 0.04| (131)004/ 0.03| (132)003/ 0.04| (133)002/ 0.02| (134)001/ 0.03| (135)006/ 0.03| (136)002/ 0.03| (137)003/ 0.03| (138)002/ 0.02| (139)002/ 0.03| (140)002/ 0.02| (141)001/ 0.03| (142)002/ 0.03| (143)006/ 0.04| (144)001/ 0.03| (145)002/ 0.02| (146)002/ 0.04| (147)004/ 0.03| (148)001/ 0.03| (149)00